In [1]:
import sys
from pathlib import Path

# Add user_data directory to path for crypto_analysis package
user_data_path = Path.cwd().parent
data_dir = user_data_path / "data" / "binance"
sys.path.insert(0, str(user_data_path))

In [2]:
from crypto_analysis import (
    DatasetBuilder,
    DataPreprocessor,
    SequenceValidator,
    SignalDataset,
    create_sequences,
    LSTMSignalPredictor,
    ModelConfig,
    WeightedSignalLoss,
    Trainer,
    TrainingConfig,
    Predictor,
    LSTMMetaheuristicOptimizer,
)


In [3]:
# 1. Build dataset
builder = DatasetBuilder(data_dir=data_dir, period_hours=4, n_workers=15, signal_shift=4, output_mode="raw")
df = builder.build('DOGE', 
                   threshold_pct=1.5,
                   hyperopt=False,
                   verbose=True)

Generating target signals for DOGE...
Initial index (0): entry=140, exit=140
Found optimal index (0): entry=140, exit=140
Processing 33 indicators with 15 workers...
Signal shift: 4 steps (indicators predict t+4)
  [RSI] Grid search optimization...
  [MACD] Grid search optimization...
  [STOCH] Grid search optimization...
  [STOCHRSI] Grid search optimization...
  [CCI] Grid search optimization...
  [MFI] Grid search optimization...
  [WILLR] Grid search optimization...
  [CMO] Grid search optimization...
  [ADX] Grid search optimization...
  [MOM] Grid search optimization...
  [ROC] Grid search optimization...
  [TRIX] Grid search optimization...
  [ULTOSC] Grid search optimization...
  [APO] Grid search optimization...
  [PPO] Grid search optimization...
  [TRIX] Grid search done - score: 16
  [BOP] Grid search optimization...
Completed 1/33: TRIX
  [BOP] Grid search done - score: 0
  [AROON] Grid search optimization...
Completed 2/33: BOP
  [ROC] Grid search done - score: 48
  [AROO

In [4]:
# Optimize
optimizer = LSTMMetaheuristicOptimizer(
    df=df,
    pop_size=10,
    iterations=50,
    n_workers=10,
    np_neighbors=2,
    pf_max=0.15,
    epochs_per_eval=100,
    checkpoint_interval=5,
    elitist_selection=True,
    elitist_constant=0.35,
    verbose=True,
    enable_logging=True,
    checkpoint_dir="checkpoints_lstm_ti",
)
result = optimizer.optimize()

print(f"Best fitness: {result.best_fitness}")
print(f"Selected features ({result.n_features_selected}): {result.selected_features}")
print(f"Best params: {result.best_params}")


LSTMMetaheuristicOptimizer (APO) initialized:
  - DataFrame mode: raw
  - Feature columns: 49
  - Hyperparameters: 11
  - Total dimension: 60
  - Population size: 10
  - Iterations: 50
  - Workers: 10
  - APO np_neighbors: 2
  - APO pf_max: 0.15
  - Elitist selection: True
  - Elitist constant: 0.35
  - Correlation vector: min=0.0000, max=1.0000, mean=0.0279

Starting APO Metaheuristic Optimization
Run ID: ed3793a4

Evaluating initial population...
iter:-1 indv:1 fitness:0.0000 features:25
iter:-1 indv:4 fitness:0.0000 features:26
iter:-1 indv:6 fitness:0.0000 features:20
iter:-1 indv:0 fitness:0.0000 features:18
iter:-1 indv:7 fitness:0.0000 features:21
iter:-1 indv:8 fitness:0.0000 features:27
iter:-1 indv:3 fitness:0.0000 features:25
iter:-1 indv:2 fitness:0.0000 features:23
iter:-1 indv:5 fitness:0.0000 features:26
iter:-1 indv:9 fitness:0.0000 features:22

------------------------------------------------------------------------
INIT - Best Individual Per-Class Metrics (fitness: 0.

KeyboardInterrupt: 